In [ ]:
import spotpy

# Easy Tutorial

In [ ]:
from spotpy import analyser                                     # Load the Plotting extension 
from spotpy.examples.spot_setup_rosenbrock import spot_setup    # Import the two dimensional Rosenbrock example

In [ ]:
# Give Monte Carlo algorithm the example setup and saves results in a RosenMC.csv file
sampler = spotpy.algorithms.mc(spot_setup(), dbname='RosenMC', dbformat='csv')

In [ ]:
# sample with the implemented Monte Carlo algorithm
sampler.sample(100000)                # Sample 100,000 parameter combinations
results=sampler.getdata()             # Get the results of the sampler

In [ ]:
# what the algorithm has done during the 100.000 iterations
spotpy.analyser.plot_parameterInteraction(results)  

In [ ]:
print(spotpy.analyser.get_best_parameterset(results))

# Link BMI_CFE with Spotpy

In [ ]:
import os
#from netCDF4 import Dataset
#from pathlib import Path
import time
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt


In [ ]:
import sys

import bmi_cfe
import cfe

## set up spotpy class

In [ ]:
class spotpy_setup(object): 

    def __init__(self): 
        # setup calibration parameters
        self.params = [spotpy.parameter.Uniform('bb',0,21.94,optguess=16),
                       spotpy.parameter.Uniform('smcmax',0.20554,1,optguess=0.439), #maybe max = 0.5
                       spotpy.parameter.Uniform('satdk',0,0.000726,optguess=3.38e-6),
                       spotpy.parameter.Uniform('slop',0,1,optguess=0.01),
                       spotpy.parameter.Uniform('max_gw_storage',0.01,0.25,optguess=0.01),
                       spotpy.parameter.Uniform('expon',1,8,optguess=6.0),
                       spotpy.parameter.Uniform('Cgw',1.8e-6,1.8e-3),
                       spotpy.parameter.Uniform('K_lf',0,0.1),
                       spotpy.parameter.Uniform('K_nash',0,0.1),
                       ]
    
        #Load test comparison data (streamflow) from usgs data
        data = pd.read_csv('/Users/Sophie/Desktop/CUAHSI/CFE_Calibration/01022500-usgs-hourly.csv')
        self.obs_data = data['QObs(mm/h)'].values
        self.eval_dates = data['date'].values
        print('###--------- usgs start date: ' + self.eval_dates[0] + '.---------')
        print('###--------- usgs end date: ' + self.eval_dates[-1] + '.---------')

        self.cfemodel = bmi_cfe.BMI_CFE('/Users/Sophie/Desktop/CUAHSI/CFE_Calibration/cal_test_config_cfe.json')
        print('###--------model succesfully setup----------###')
        self.cfemodel.initialize()
        print('###--------model succesfully initialized----------###')

        with open(self.cfemodel.forcing_file, 'r') as f:
            self.df_forcing = pd.read_csv(f)

        usgs_start_idx = np.where(self.df_forcing['date']==self.eval_dates[0])
        self.df_forcing = self.df_forcing.iloc[usgs_start_idx[0][0]:,:]

        print(f"###----- forcing_file loaded:{self.cfemodel.forcing_file}. -----###")
        print('###------nldas start date: ' + self.df_forcing['date'].values[0]+ "-----###")
        print('###------nldas end date: ' + self.df_forcing['date'].values[-1]+"-----###")

        print('###-----after defining start date nldas length: ' +  str(len(self.df_forcing['date'].values))+"------###")

        nldas_end_idx = np.where(self.eval_dates==self.df_forcing['date'].values[-1])
        self.eval_dates = self.eval_dates[:nldas_end_idx[0][0]+1]
        self.obs_data = self.obs_data[:nldas_end_idx[0][0]+1]
        print('###---------- after defining usgs end date usgs length: ' +  str(len(self.obs_data)) + '.---------')

        
    def parameters(self):
        return spotpy.parameter.generate(self.params)
        
    def simulation(self,vector):
        self.cfemodel = bmi_cfe.BMI_CFE('/Users/Sophie/Desktop/CUAHSI/CFE_Calibration/cal_test_config_cfe.json')
        print('###--------model succesfully setup----------###')
        self.cfemodel.initialize()
        print('###--------model succesfully initialized----------###')

        #Read in Meteorological forcing
        with open(self.cfemodel.forcing_file, 'r') as f:
            self.df_forcing = pd.read_csv(f)

        usgs_start_idx = np.where(self.df_forcing['date']==self.eval_dates[0])
        self.df_forcing = self.df_forcing.iloc[usgs_start_idx[0][0]:,:]

        print(f"###----- forcing_file loaded:{self.cfemodel.forcing_file}. -----###")
        print('###------nldas start date: ' + self.df_forcing['date'].values[0]+ "-----###")
        print('###------nldas end date: ' + self.df_forcing['date'].values[-1]+"-----###")

        print('###-----after defining start date nldas length: ' +  str(len(self.df_forcing['date'].values))+"------###")

        nldas_end_idx = np.where(self.eval_dates==self.df_forcing['date'].values[-1])
        self.eval_dates = self.eval_dates[:nldas_end_idx[0][0]+1]
        self.obs_data = self.obs_data[:nldas_end_idx[0][0]+1]
        print('###---------- after defining usgs end date usgs length: ' +  str(len(self.obs_data)) + '.---------')

        #test.cfemodel.cfe_model = cfe.CFE()
        
        self.generated_param = vector
        print(f"###----------- parameters generated: {self.generated_param}.--------###")
        self.cfemodel.soil_params['bb']=vector[0]
        self.cfemodel.soil_params['smcmax']=vector[1]
        self.cfemodel.soil_params['satdk']=vector[2]
        self.cfemodel.soil_params['slop']=vector[3]
        self.cfemodel.max_gw_storage==vector[4]
        self.cfemodel.expon=vector[5]
        self.cfemodel.Cgw=vector[6]
        self.cfemodel.K_lf=vector[7]
        self.cfemodel.K_nash=vector[8]

        self.outputs=self.cfemodel.get_output_var_names()
        self.output_lists = {output:[] for output in self.outputs}

        for precip, pet in zip(self.df_forcing['total_precipitation'],self.df_forcing['potential_evaporation']):
            #print(f"###----------loaded precip, pet: {precip},{pet}.------------###")
            #sys.exit(1)
            self.cfemodel.set_value('atmosphere_water__time_integral_of_precipitation_mass_flux', precip)
            self.cfemodel.set_value('water_potential_evaporation_flux', pet)
            self.cfemodel.update()
            
        # for precip in self.df_forcing['total_precipitation']:
        #     self.cfemodel.set_value('atmosphere_water__time_integral_of_precipitation_mass_flux', precip)
        #     self.cfemodel.update()
        
            for output in self.outputs:
                self.output_lists[output].append(self.cfemodel.get_value(output))
           
        self.cfemodel.finalize()

        print(f'###----------output length: {len(self.output_lists["land_surface_water__runoff_volume_flux"])}.---------###')

        return self.output_lists["land_surface_water__runoff_volume_flux"]

    def evaluation(self,evaldates=False):
        if evaldates:
            self.eval_dates_output = [pd.Timestamp(test.eval_dates[i]) for i in range(len(test.eval_dates))]
            return self.eval_dates_output
        else:
            print(f"length of obs_data: {len(self.obs_data)}.")
            return self.obs_data

    def objectivefunction(self,simulation,evaluation, params=None):
        self.obj_function = spotpy.objectivefunctions.kge(evaluation,simulation)
        return self.obj_function

## Evaluate and Plot

In [ ]:
test = spotpy_setup()
sampler = spotpy.algorithms.dds(test,dbname='TestDDS',dbformat='csv')

In [ ]:
sampler.sample(10)

In [ ]:
results = sampler.getdata()

In [ ]:
print(spotpy.analyser.get_best_parameterset(results))

In [ ]:
spotpy.analyser.plot_parametertrace(results)

In [ ]:
evaluation = test.evaluation()
evaldates= test.evaluation(evaldates=True)

spotpy.analyser.plot_bestmodelruns(results,evaluation,algorithms='dds',dates=evaldates, ylabel='Stream Flow')